In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import simpleaudio as sa
import numpy as np
import os

import IPython.display as ipd

In [3]:
from pySynth.io.IO import ReadMIDI
from pySynth.control.stream import EffectChain, MIDIStream
from pySynth.synth.effect import *
from pySynth.synth.filter import *
from pySynth.synth.synthesizer import *

In [4]:
midi_filepath = "D:\\Documents\\2021Spring\\DSP_Final\\midi\\test.mid"

midi_func = ReadMIDI(midi_filepath)
midi_ls = midi_func()
print(midi_ls)

# >>> [[60, 0.0, 0.5], [64, 0.5, 1.0], [67, 1.0, 1.5], [64, 2.0, 2.5], [62, 2.5, 2.75], 
# [60, 2.75, 3.0], [62, 3.0, 3.125], [60, 3.25, 3.375], [60, 3.5, 4.0], [69, 4.0, 4.5], 
# [67, 4.5, 5.0], [62, 5.0, 5.25], [64, 5.25, 5.5], [62, 5.5, 5.75], [62, 5.75, 6.0], 
# [64, 6.0, 6.25], [60, 6.5, 7.0], [62, 7.0, 7.5], [60, 7.5, 8.0]]

[[60, 0.0, 0.5], [64, 0.5, 1.0], [67, 1.0, 1.5], [64, 2.0, 2.5], [62, 2.5, 2.75], [60, 2.75, 3.0], [62, 3.0, 3.125], [60, 3.25, 3.375], [60, 3.5, 4.0], [69, 4.0, 4.5], [67, 4.5, 5.0], [62, 5.0, 5.25], [64, 5.25, 5.5], [62, 5.5, 5.75], [62, 5.75, 6.0], [64, 6.0, 6.25], [60, 6.5, 7.0], [62, 7.0, 7.5], [60, 7.5, 8.0]]


# Midi Input

In [6]:
fs = 44100
normalization_factor = 1
stream = MIDIStream("sine")
audio = stream.render(midi_ls)
audio = ((audio) / normalization_factor * 32767).astype(np.int16)

ipd.Audio(audio, rate = 44100)

# One Effect Chain

In [7]:
effects = EffectChain(
#     Reverb(1,1,"hall"),
    Delay(delayTime = 0.25),
#     Chorus(),
#     Vibrato(),
    LowPass(Low_Band=800)
)

fs = 44100

stream = MIDIStream("sine")
audio = stream.render(midi_ls)
audio = effects(audio)

ipd.Audio(audio, rate = 44100)

# Multiple Effect Chains & AddSynth

In [9]:
effects1 = EffectChain(
    Reverb(1,1,"hall"),
    Delay(delayTime = 0.25),
)

effects2 =  EffectChain(
    Chorus(),
    BandPass(Low_Band=400)
)

fs = 44100

stream = MIDIStream("triangle")
audio = stream.render(midi_ls)
audio1 = effects1(audio)
audio2 = effects2(audio)

synth = AddSynth(
    audio1,
    audio2)

audio = synth()

ipd.Audio(audio, rate = 44100)

# Misc Code; not for use

In [11]:
frequency = 440  # Our played note will be 440 Hz
fs = 44100  # 44100 samples per second
seconds = 2  # Note duration of 3 seconds

# Generate array with seconds*sample_rate steps, ranging between 0 and seconds
t = np.linspace(0, seconds, seconds * fs, False)

# Generate a 440 Hz sine wave
note = np.sin(frequency * t * 2 * np.pi)

# Ensure that highest value is in 16-bit range
audio = note * (2**15 - 1) / np.max(np.abs(note))
# Convert to 16-bit data
audio = audio.astype(np.int16)
print(max(audio))
# # Start playback
# play_obj = sa.play_buffer(audio, 1, 2, fs)

# # Wait for playback to finish before exiting
# play_obj.wait_done()

32766


In [37]:
frequency = 880  # Our played note will be 440 Hz
fs = 44100  # 44100 samples per second
seconds = 2  # Note duration of 3 seconds

# Generate array with seconds*sample_rate steps, ranging between 0 and seconds
t = np.linspace(0, seconds, seconds * fs, False)

# Generate a 440 Hz sine wave
note2 = np.sin(frequency * t * 2 * np.pi)

# Ensure that highest value is in 16-bit range
audio2 = note2 * (2**15 - 1) / np.max(np.abs(note2))
# Convert to 16-bit data
audio2 = audio2.astype(np.int16)

# # Start playback
# play_obj = sa.play_buffer(audio, 1, 2, fs)

# # Wait for playback to finish before exiting
# play_obj.wait_done()

In [38]:
play_obj = sa.play_buffer(audio+audio2, 1, 2, fs)
play_obj.wait_done()

In [15]:
play_obj = sa.play_buffer(waveform_integers, 1, 2, sps)

In [ ]:
attack = np.linspace(0, 1, attack_length)
decay = np.linspace(1, sustain_level, decay_length)
sustain = np.full((sustain_length, ), sustain_level)
release = np.linspace(sustain_level, 0, release_length)

In [14]:
osc = OSC("sine", 440,1)

In [15]:
from pySynth.synth.oscillator import SineOscillator, OSC

In [17]:

# osc = SineOscillator(freq=440)
osc = OSC("sine", 440, 2)

# iter(osc)
# wav = [next(osc) for _ in range(44100 * 2)] # 4 Seconds
# audio = (np.array(wav) * 32766).astype(np.int16)

audio = (np.array(osc()) * 32766).astype(np.int16)


# Start playback
play_obj = sa.play_buffer(audio, 1, 2, 44100)

# Wait for playback to finish before exiting
play_obj.wait_done()

In [64]:
class AddSynth:
    def __init__(self, *streams):
        self.streams = streams
        for stream in streams:
            print(stream)
    
    def __call__(self):
        return np.mean([stream() for stream in self.streams], axis = 0)

class WaveTable:
    def __init__(self, *streams):
        self.streams = streams
        for stream in streams:
            assert isinstance(stream, OSC) or isinstance(stream, Stream)
    
    def __call__(self):
        return np.hstack([stream() for stream in self.streams])

In [81]:
add = AddSynth(
    OSC("sine", 440, 2),
    OSC("square", 440, 2),
    OSC("triangle", 440, 2),
)

wt = WaveTable(
    OSC("sine", 440, 0.01),
    OSC("square", 440, 0.01),
)

In [82]:
audio = (np.array(add()) * 32766 * 0.1).astype(np.int16)
play_obj = sa.play_buffer(audio, 1, 2, 44100)

# Wait for playback to finish before exiting
play_obj.wait_done()

In [77]:
audio = (np.array(wt()) * 32766 * 0.1).astype(np.int16)
play_obj = sa.play_buffer(audio, 1, 2, 44100)

# Wait for playback to finish before exiting
play_obj.wait_done()